In [7]:
import pandas as pd
df=pd.read_csv('creditcard.csv')
df.head()

/home/self-made-lol/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

In [8]:
df.shape

(284807, 31)

In [9]:
df.isnull().sum().count()

31

In [10]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

# cross validation like KFOLD and Hyper parameter Tuning

In [11]:
### independent and dependent features
X = df.drop("Class",axis=1)
y = df.Class

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV

In [13]:
log_class = LogisticRegression()
grid={'C':10.0 **np.arange(-2,3), 'penalty':['l1','l2']}
cv = KFold(n_splits=5,random_state=None,shuffle=False)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X,y, train_size=0.7)

In [15]:
clf = GridSearchCV(log_class,grid,cv=cv, n_jobs=-1, scoring='f1_macro')
clf.fit(X_train,y_train)

/home/self-made-lol/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [16]:
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85235    64]
 [   44   100]]
0.998735999438222
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.61      0.69      0.65       144

    accuracy                           1.00     85443
   macro avg       0.80      0.85      0.82     85443
weighted avg       1.00      1.00      1.00     85443



In [17]:
class_weight=dict({0:1,1:100})

In [18]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [19]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85292     7]
 [   39   105]]
0.9994616293903538
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.94      0.73      0.82       144

    accuracy                           1.00     85443
   macro avg       0.97      0.86      0.91     85443
weighted avg       1.00      1.00      1.00     85443



# Under Sampling

In [20]:
from collections import Counter
from imblearn.under_sampling import NearMiss
ns=NearMiss(0.8)
X_train_ns,y_train_ns=ns.fit_sample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

/home/self-made-lol/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fit Counter({0: 199016, 1: 348})
The number of classes after fit Counter({0: 435, 1: 348})


In [21]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [22]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[60277 25022]
 [   10   134]]
0.7070327586812261
              precision    recall  f1-score   support

           0       1.00      0.71      0.83     85299
           1       0.01      0.93      0.01       144

    accuracy                           0.71     85443
   macro avg       0.50      0.82      0.42     85443
weighted avg       1.00      0.71      0.83     85443



# Over Sampling

In [23]:
from imblearn.over_sampling import RandomOverSampler

In [24]:
os=RandomOverSampler(0.5)
X_train_ns,y_train_ns=os.fit_sample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

/home/self-made-lol/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fit Counter({0: 199016, 1: 348})
The number of classes after fit Counter({0: 199016, 1: 99508})


In [25]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [26]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85288    11]
 [   32   112]]
0.9994967405170698
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.91      0.78      0.84       144

    accuracy                           1.00     85443
   macro avg       0.96      0.89      0.92     85443
weighted avg       1.00      1.00      1.00     85443



# SmoteTomek

In [27]:
from imblearn.combine import SMOTETomek

In [28]:
os=SMOTETomek(0.5)
X_train_ns,y_train_ns=os.fit_sample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

/home/self-made-lol/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fit Counter({0: 199016, 1: 348})
The number of classes after fit Counter({0: 198081, 1: 98573})


In [29]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [30]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85271    28]
 [   23   121]]
0.999403110845827
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.81      0.84      0.83       144

    accuracy                           1.00     85443
   macro avg       0.91      0.92      0.91     85443
weighted avg       1.00      1.00      1.00     85443



# Ensemble Techniques

In [31]:
from imblearn.ensemble import EasyEnsembleClassifier

In [32]:
easy = EasyEnsembleClassifier()
easy.fit(X_train,y_train)

EasyEnsembleClassifier()

In [33]:
y_pred=easy.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[82629  2670]
 [    9   135]]
0.9686457638425617
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     85299
           1       0.05      0.94      0.09       144

    accuracy                           0.97     85443
   macro avg       0.52      0.95      0.54     85443
weighted avg       1.00      0.97      0.98     85443



# Ensemble Techniques gives Bad Precision ... Now its all upto you .. you can use Isolation forest for more accuracy